In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy

import socketio
import asyncio

from flask import Flask, render_template
from flask_socketio import SocketIO, send, emit

In [ ]:
sio = socketio.AsyncClient()
part1_model = tf.keras.models.load_model('models/part1')

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

In [ ]:
@sio.on('to part1', namespace="/net")
async def got_input(data):
    print("Received data in part1")
    inpt = tf.convert_to_tensor(data["server"])
    out = part1_model(inpt).numpy().tolist()
    await sio.emit('from node', data=("part1", out), namespace="/net")

@sio.on('connect', namespace="/net")
def connect():
    print("I'm connected!")

In [ ]:
await sio.connect('http://localhost:5000', namespaces=['/recv_model'], auth={"name": "part1"})
await sio.wait()

In [ ]:
if __name__ == '__main__':
    socketio.run(app, port=5001)